### Step1: Data conversion
Read the BAG dataset, convert it to a geopaquet.

In [1]:
from pathlib import Path
from datetime import datetime

# dask_geopandas, dask-expr, pyogrio
import dask_geopandas as dg

In [2]:
# Get metadata
data_BAG = dg.read_file('./download/bag-light-NL.gpkg', npartitions=50)
data_BAG

,rdf_seealso,identificatie,bouwjaar,status,gebruiksdoel,oppervlakte_min,oppervlakte_max,aantal_verblijfsobjecten,geometry
npartitions=50,,,,,,,,,
0,string,string,int64,string,string,float64,float64,int64,geometry
217153,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
10640497,...,...,...,...,...,...,...,...,...
10857623,...,...,...,...,...,...,...,...,...


In [3]:
# Create a path to save the coverted data
path_BAG = Path('./data/bag.parquet/')
path_BAG.mkdir(exist_ok=True, parents=True)

In [4]:
# This does not work, complains about the data type of the geometry column
# data_BAG.to_parquet(path_BAG)
# data_BAG[['rdf_seealso', 'identificatie', 'bouwjaar', 'status', 'gebruiksdoel',
#        'oppervlakte_min', 'oppervlakte_max', 'aantal_verblijfsobjecten']].to_parquet(path_BAG) # This works because there is no geometry column

for i in range(data_BAG.npartitions):
    data_partition = data_BAG.get_partition(i).compute()
    data_partition.to_parquet(path_BAG / f'bag_{i}.parquet')

### Compair performace of gpkg and geopaquet

Test getting two columns from the BAG dataset, and then compute the first partition. Reading from parquet is almost 50 times faster than reading from geopackage.

In [5]:
%%timeit
data_BAG_gpks = dg.read_file('./download/bag-light-NL.gpkg', npartitions=50, columns=['bouwjaar', 'status'])
data_BAG_slice = data_BAG_gpks.get_partition(0).compute()

26.7 s ± 566 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit
data_BAG_parque = dg.read_parquet('./data/bag.parquet/', columns=['bouwjaar', 'status'])
data_BAG_parque.get_partition(0).compute()

66.4 ms ± 2.89 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Convert KNMI station data

In [7]:
import pandas as pd
import xarray as xr
import numpy as np
import datetime

In [8]:
df_stations = pd.read_csv('./download/KNMI/knmistations.csv')
df_stations = df_stations.rename(columns={'STN': 'station'}).set_index('station')
df_stations

,STARTT,STOPT,LOCATIE,HOOGTE,POS_X,POS_Y,POS_NB,POS_OL
station,,,,,,,,
201,20090313,99991231,platformD15-FA-1,42.7,-4.5,707.1,54.3,2.9
203,20091109,99991231,platformP11-B,41.8,15.7,487.6,52.4,3.3
204,20061206,99991231,platformK14-FA-1C,41.8,37.6,588.3,53.3,3.6
205,20090313,99991231,platformA12-CPP,48.4,55.0,825.1,55.4,3.8
206,20061206,99991231,platformF16-A,43.4,65.1,682.0,54.1,4.0
...,...,...,...,...,...,...,...,...
370,19510101,99991231,Eindhoven,22.6,154.3,384.4,51.4,5.4
375,19510201,99991231,Volkel,22.0,177.1,407.7,51.7,5.7
377,19990501,99991231,Ell,30.0,181.2,356.3,51.2,5.8


In [9]:
ds = xr.Dataset.from_dataframe(df_stations)
ds

<xarray.Dataset>
Dimensions:  (station: 63)
Coordinates:
  * station  (station) int64 201 203 204 205 206 207 ... 356 370 375 377 380 391
Data variables:
    STARTT   (station) int64 20090313 20091109 20061206 ... 19060101 19900618
    STOPT    (station) int64 99991231 99991231 99991231 ... 99991231 99991231
    LOCATIE  (station) object 'platformD15-FA-1' 'platformP11-B' ... 'Arcen'
    HOOGTE   (station) float64 42.7 41.8 41.8 48.4 43.4 ... 22.0 30.0 114.3 19.5
    POS_X    (station) float64 -4.5 15.7 37.6 55.0 ... 177.1 181.2 181.3 211.1
    POS_Y    (station) float64 707.1 487.6 588.3 825.1 ... 356.3 323.9 390.0
    POS_NB   (station) float64 54.3 52.4 53.3 55.4 54.1 ... 51.7 51.2 50.9 51.5
    POS_OL   (station) float64 2.9 3.3 3.6 3.8 4.0 5.0 ... 5.4 5.7 5.8 5.8 6.2

In [10]:
# Convert data type
ds['LOCATIE'] = ds['LOCATIE'].astype(str)

# Convert RD coordinates from km to meters
ds['POS_X'] = ds['POS_X'] * 1000
ds['POS_Y'] = ds['POS_Y'] * 1000

ds

<xarray.Dataset>
Dimensions:  (station: 63)
Coordinates:
  * station  (station) int64 201 203 204 205 206 207 ... 356 370 375 377 380 391
Data variables:
    STARTT   (station) int64 20090313 20091109 20061206 ... 19060101 19900618
    STOPT    (station) int64 99991231 99991231 99991231 ... 99991231 99991231
    LOCATIE  (station) <U20 'platformD15-FA-1' 'platformP11-B' ... 'Arcen'
    HOOGTE   (station) float64 42.7 41.8 41.8 48.4 43.4 ... 22.0 30.0 114.3 19.5
    POS_X    (station) float64 -4.5e+03 1.57e+04 ... 1.813e+05 2.111e+05
    POS_Y    (station) float64 7.071e+05 4.876e+05 ... 3.239e+05 3.9e+05
    POS_NB   (station) float64 54.3 52.4 53.3 55.4 54.1 ... 51.7 51.2 50.9 51.5
    POS_OL   (station) float64 2.9 3.3 3.6 3.8 4.0 5.0 ... 5.4 5.7 5.8 5.8 6.2

In [11]:
# Load the knmi file for station 280 as a pandas dataframe

# Get the column names
header = pd.read_csv('./download/KNMI/knmi_stn280_example_file.txt', skiprows=32, nrows=0, delimiter=',')
col_names = [col.replace(' ', '') for col in header.columns] # Remove the spaces from the column names
col_names = [col.replace('#', '') for col in col_names] # Remove the # from the column names

# skip 32 rows since they are comments
# keep row 31 as header
# remove row 32 since it only has a hashtag
df_one_stn = pd.read_csv('./download/KNMI/knmi_stn280_example_file.txt', skiprows=34, header=0)

# Add the column names
df_one_stn.columns = col_names

df_one_stn

,STN,YYYYMMDD,HH,DD,FH,FF,FX,T,T10,TD,...,VV,N,U,WW,IX,M,R,S,O,Y
0,280,20080101,2,210,0,10,10,-33,,-33,...,0,9,100,35,7,1,0,0,0,1
1,280,20080101,3,0,0,0,10,-31,,-31,...,0,9,100,35,7,1,0,0,0,1
2,280,20080101,4,0,0,0,10,-29,,-29,...,2,9,100,35,7,1,0,0,0,1
3,280,20080101,5,990,10,10,20,-22,,-23,...,17,8,99,20,7,1,0,0,0,1
4,280,20080101,6,990,10,10,20,-19,-35,-19,...,12,8,100,10,7,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960,280,20080213,16,360,20,20,40,35,,17,...,36,8,88,10,7,0,0,0,0,0
961,280,20080213,17,20,20,30,40,27,,14,...,36,8,91,10,7,0,0,0,0,0
962,280,20080213,18,20,30,30,60,20,20,11,...,27,8,94,10,7,0,0,0,0,0
963,280,20080213,19,30,30,20,50,21,,12,...,30,8,94,10,7,0,0,0,0,0


In [12]:
# Replace blanks with NaN
for col in ['T10', 'WW']:
    df_one_stn[col] = df_one_stn[col].replace(r'^\s*$', float('nan'), regex=True).astype(float) # Replace empty strings with nan, then convert to float

In [13]:
hours = pd.to_datetime(
    (df_one_stn["HH"]-1).astype(str).str.zfill(2) + ":00", format="%H:%M"
).dt.strftime("%H:%M")
hours

0      01:00
1      02:00
2      03:00
3      04:00
4      05:00
       ...  
960    15:00
961    16:00
962    17:00
963    18:00
964    19:00
Name: HH, Length: 965, dtype: object

In [14]:
dates = pd.to_datetime(df_one_stn["YYYYMMDD"], format="%Y%m%d").dt.date.astype(str)

# Subtract 1 hour from the hour column to avoid the 24:00 hour.
# The description of the HH field has been updated to reflect this
hours = pd.to_datetime(
    (df_one_stn["HH"]-1).astype(str).str.zfill(2) + ":00", format="%H:%M"
).dt.strftime("%H:%M") 

datetimes = pd.to_datetime(dates + " " + hours)

datetimes

0     2008-01-01 01:00:00
1     2008-01-01 02:00:00
2     2008-01-01 03:00:00
3     2008-01-01 04:00:00
4     2008-01-01 05:00:00
              ...        
960   2008-02-13 15:00:00
961   2008-02-13 16:00:00
962   2008-02-13 17:00:00
963   2008-02-13 18:00:00
964   2008-02-13 19:00:00
Length: 965, dtype: datetime64[ns]

In [15]:
df_one_stn['time'] = datetimes
ds_one_stn = df_one_stn.set_index('time').to_xarray()
ds_one_stn

<xarray.Dataset>
Dimensions:   (time: 965)
Coordinates:
  * time      (time) datetime64[ns] 2008-01-01T01:00:00 ... 2008-02-13T19:00:00
Data variables: (12/25)
    STN       (time) int64 280 280 280 280 280 280 ... 280 280 280 280 280 280
    YYYYMMDD  (time) int64 20080101 20080101 20080101 ... 20080213 20080213
    HH        (time) int64 2 3 4 5 6 7 8 9 10 11 ... 12 13 14 15 16 17 18 19 20
    DD        (time) int64 210 0 0 990 990 0 40 80 ... 50 40 10 360 20 20 30 30
    FH        (time) int64 0 0 0 10 10 10 10 20 20 ... 20 20 20 20 20 30 30 30
    FF        (time) int64 10 0 0 10 10 0 10 30 20 ... 20 20 10 20 30 30 20 30
    ...        ...
    IX        (time) int64 7 7 7 7 7 7 7 7 7 7 7 7 7 ... 7 7 7 7 7 7 7 7 7 7 7 7
    M         (time) int64 1 1 1 1 0 0 1 1 1 1 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0
    R         (time) int64 0 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0
    S         (time) int64 0 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0
    O         (time) int64 0 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0
    Y         (time) int64 1 1 1 1 0 0 1 1 1 1 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0

In [16]:
stn = np.unique(ds_one_stn['STN'].values)
assert stn.shape[0]==1

ds_one_stn = ds_one_stn.expand_dims('station').assign_coords(station=stn)
ds_one_stn

<xarray.Dataset>
Dimensions:   (time: 965, station: 1)
Coordinates:
  * time      (time) datetime64[ns] 2008-01-01T01:00:00 ... 2008-02-13T19:00:00
  * station   (station) int64 280
Data variables: (12/25)
    STN       (station, time) int64 280 280 280 280 280 ... 280 280 280 280 280
    YYYYMMDD  (station, time) int64 20080101 20080101 ... 20080213 20080213
    HH        (station, time) int64 2 3 4 5 6 7 8 9 ... 13 14 15 16 17 18 19 20
    DD        (station, time) int64 210 0 0 990 990 0 40 ... 10 360 20 20 30 30
    FH        (station, time) int64 0 0 0 10 10 10 10 ... 20 20 20 20 30 30 30
    FF        (station, time) int64 10 0 0 10 10 0 10 ... 20 10 20 30 30 20 30
    ...        ...
    IX        (station, time) int64 7 7 7 7 7 7 7 7 7 7 ... 7 7 7 7 7 7 7 7 7 7
    M         (station, time) int64 1 1 1 1 0 0 1 1 1 1 ... 0 0 0 0 0 0 0 0 0 0
    R         (station, time) int64 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    S         (station, time) int64 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    O         (station, time) int64 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    Y         (station, time) int64 1 1 1 1 0 0 1 1 1 1 ... 0 0 0 0 0 0 0 0 0 0

In [17]:
ds_output = ds.merge(ds_one_stn, combine_attrs='override')
ds_output

<xarray.Dataset>
Dimensions:   (station: 63, time: 965)
Coordinates:
  * station   (station) int64 201 203 204 205 206 207 ... 370 375 377 380 391
  * time      (time) datetime64[ns] 2008-01-01T01:00:00 ... 2008-02-13T19:00:00
Data variables: (12/33)
    STARTT    (station) int64 20090313 20091109 20061206 ... 19060101 19900618
    STOPT     (station) int64 99991231 99991231 99991231 ... 99991231 99991231
    LOCATIE   (station) <U20 'platformD15-FA-1' 'platformP11-B' ... 'Arcen'
    HOOGTE    (station) float64 42.7 41.8 41.8 48.4 ... 22.0 30.0 114.3 19.5
    POS_X     (station) float64 -4.5e+03 1.57e+04 ... 1.813e+05 2.111e+05
    POS_Y     (station) float64 7.071e+05 4.876e+05 ... 3.239e+05 3.9e+05
    ...        ...
    IX        (station, time) float64 nan nan nan nan nan ... nan nan nan nan
    M         (station, time) float64 nan nan nan nan nan ... nan nan nan nan
    R         (station, time) float64 nan nan nan nan nan ... nan nan nan nan
    S         (station, time) float64 nan nan nan nan nan ... nan nan nan nan
    O         (station, time) float64 nan nan nan nan nan ... nan nan nan nan
    Y         (station, time) float64 nan nan nan nan nan ... nan nan nan nan

In [18]:
ds_output

<xarray.Dataset>
Dimensions:   (station: 63, time: 965)
Coordinates:
  * station   (station) int64 201 203 204 205 206 207 ... 370 375 377 380 391
  * time      (time) datetime64[ns] 2008-01-01T01:00:00 ... 2008-02-13T19:00:00
Data variables: (12/33)
    STARTT    (station) int64 20090313 20091109 20061206 ... 19060101 19900618
    STOPT     (station) int64 99991231 99991231 99991231 ... 99991231 99991231
    LOCATIE   (station) <U20 'platformD15-FA-1' 'platformP11-B' ... 'Arcen'
    HOOGTE    (station) float64 42.7 41.8 41.8 48.4 ... 22.0 30.0 114.3 19.5
    POS_X     (station) float64 -4.5e+03 1.57e+04 ... 1.813e+05 2.111e+05
    POS_Y     (station) float64 7.071e+05 4.876e+05 ... 3.239e+05 3.9e+05
    ...        ...
    IX        (station, time) float64 nan nan nan nan nan ... nan nan nan nan
    M         (station, time) float64 nan nan nan nan nan ... nan nan nan nan
    R         (station, time) float64 nan nan nan nan nan ... nan nan nan nan
    S         (station, time) float64 nan nan nan nan nan ... nan nan nan nan
    O         (station, time) float64 nan nan nan nan nan ... nan nan nan nan
    Y         (station, time) float64 nan nan nan nan nan ... nan nan nan nan

In [19]:
# Manually convert descriptions text to a dictionary
# The descriptions of HH has been changed accordingly to avoid 24:00 hour
description_text = {
    "YYYYMMDD": "datum (YYYY=jaar,MM=maand,DD=dag);",
    "HH": "tijd (HH=uur, UT.12 UT=13 MET, 14 MEZT. Uurvak 05 loopt van 05.00 UT tot 6.00 UT;",
    "DD": "Windrichting (in graden) gemiddeld over de laatste 10 minuten van het afgelopen uur (360=noord, 90=oost, 180=zuid, 270=west, 0=windstil 990=veranderlijk. Zie http://www.knmi.nl/kennis-en-datacentrum/achtergrond/klimatologische-brochures-en-boeken;",
    "FH": "Uurgemiddelde windsnelheid (in 0.1 m/s). Zie http://www.knmi.nl/kennis-en-datacentrum/achtergrond/klimatologische-brochures-en-boeken;",
    "FF": "Windsnelheid (in 0.1 m/s) gemiddeld over de laatste 10 minuten van het afgelopen uur;",
    "FX": "Hoogste windstoot (in 0.1 m/s) over het afgelopen uurvak;",
    "T": "Temperatuur (in 0.1 graden Celsius) op 1.50 m hoogte tijdens de waarneming;",
    "T10N": "Minimumtemperatuur (in 0.1 graden Celsius) op 10 cm hoogte in de afgelopen 6 uur;",
    "TD": "Dauwpuntstemperatuur (in 0.1 graden Celsius) op 1.50 m hoogte tijdens de waarneming;",
    "SQ": "Duur van de zonneschijn (in 0.1 uren) per uurvak, berekend uit globale straling  (-1 for <0.05 uur);",
    "Q": "Globale straling (in J/cm2) per uurvak;",
    "DR": "Duur van de neerslag (in 0.1 uur) per uurvak;",
    "RH": "Uursom van de neerslag (in 0.1 mm) (-1 voor <0.05 mm);",
    "P": "Luchtdruk (in 0.1 hPa) herleid naar zeeniveau, tijdens de waarneming;",
    "VV": "Horizontaal zicht tijdens de waarneming (0=minder dan 100m, 1=100-200m, 2=200-300m,..., 49=4900-5000m, 50=5-6km, 56=6-7km, 57=7-8km, ..., 79=29-30km, 80=30-35km, 81=35-40km,..., 89=meer dan 70km);",
    "N": "Bewolking (bedekkingsgraad van de bovenlucht in achtsten), tijdens de waarneming (9=bovenlucht onzichtbaar);",
    "U": "Relatieve vochtigheid (in procenten) op 1.50 m hoogte tijdens de waarneming;",
    "WW": "Weercode (00-99), visueel(WW) of automatisch(WaWa) waargenomen, voor het actuele weer of het weer in het afgelopen uur. Zie http://bibliotheek.knmi.nl/scholierenpdf/weercodes_Nederland;",
    "IX": "Weercode indicator voor de wijze van waarnemen op een bemand of automatisch station (1=bemand gebruikmakend van code uit visuele waarnemingen, 2,3=bemand en weggelaten (geen belangrijk weersverschijnsel, geen gegevens), 4=automatisch en opgenomen (gebruikmakend van code uit visuele waarnemingen), 5,6=automatisch en weggelaten (geen belangrijk weersverschijnsel, geen gegevens), 7=automatisch gebruikmakend van code uit automatische waarnemingen);",
    "M": "Mist 0=niet voorgekomen, 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming;",
    "R": "Regen 0=niet voorgekomen, 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming;",
    "S": "Sneeuw 0=niet voorgekomen, 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming;",
    "O": "Onweer 0=niet voorgekomen, 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming;",
    "Y": "IJsvorming 0=niet voorgekomen, 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming;",
}

In [20]:
for var, text in description_text.items():
    if var in ds_output:
        ds_output[var].attrs['description'] = text

In [21]:
ds_output.chunk({'station':10}).to_zarr('./data/knmi.zarr', mode='w')

In [22]:
# Check the size of the dataset
# data vars with "station" dim and "station x time" dim have the same size
# indicating compression applied to NaN values
! du -h ./data/knmi.zarr

40K	./data/knmi.zarr/IX
16K	./data/knmi.zarr/station
40K	./data/knmi.zarr/POS_NB
40K	./data/knmi.zarr/STN
40K	./data/knmi.zarr/HH
40K	./data/knmi.zarr/POS_OL
40K	./data/knmi.zarr/Y
40K	./data/knmi.zarr/FH
40K	./data/knmi.zarr/FX
40K	./data/knmi.zarr/LOCATIE
40K	./data/knmi.zarr/TD
40K	./data/knmi.zarr/P
40K	./data/knmi.zarr/POS_X
40K	./data/knmi.zarr/HOOGTE
40K	./data/knmi.zarr/M
40K	./data/knmi.zarr/SQ
40K	./data/knmi.zarr/T
40K	./data/knmi.zarr/N
40K	./data/knmi.zarr/YYYYMMDD
40K	./data/knmi.zarr/DD
40K	./data/knmi.zarr/T10
40K	./data/knmi.zarr/FF
40K	./data/knmi.zarr/Q
16K	./data/knmi.zarr/time
40K	./data/knmi.zarr/STOPT
40K	./data/knmi.zarr/STARTT
40K	./data/knmi.zarr/VV
40K	./data/knmi.zarr/RH
40K	./data/knmi.zarr/O
40K	./data/knmi.zarr/S
40K	./data/knmi.zarr/POS_Y
40K	./data/knmi.zarr/R
40K	./data/knmi.zarr/DR
40K	./data/knmi.zarr/WW
40K	./data/knmi.zarr/U
1.4M	./data/knmi.zarr
